In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import dask
import dask.dataframe as dd
from dask import delayed

from dask.diagnostics import ProgressBar

from shapely.geometry import Point
from shapely.geometry import Polygon

from datetime import datetime as dt
from datetime import timedelta


In [2]:
new_y1001_1=dd.read_csv('D:/Working_Place/Uber/FOIL2010/trip_data_1.csv',dtype={' store_and_fwd_flag': 'object'})
new_y1001_1.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2010000001,2010000001,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:34:00,1,34.0,14.05,-73.948418,40.724590,-73.926140,40.864761
1,2010000002,2010000002,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.0,9.65,-73.997414,40.736156,-73.997833,40.736168
2,2010000003,2010000003,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:07:00,1,7.0,1.63,-73.967171,40.764236,-73.956299,40.781261
3,2010000004,2010000004,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.0,26.61,-73.789757,40.646526,-74.136749,40.601543
4,2010000005,2010000005,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:28:00,2,28.0,3.15,-73.999550,40.731152,-73.977448,40.763031


In [3]:
new_y1001_1_fare=dd.read_csv('D:/Working_Place/Uber/FOIL2010/trip_fare_1.csv')
new_y1001_1_fare.head()

,medallion,hack_license,vendor_id,pickup_datetime,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,2010000001,2010000001,VTS,2010-01-01 00:00:00,CAS,34.1,0.5,0.5,0.0,0.00,35.10
1,2010000002,2010000002,VTS,2010-01-01 00:00:00,CAS,27.3,0.5,0.5,0.0,0.00,28.30
2,2010000003,2010000003,VTS,2010-01-01 00:00:00,CAS,6.9,0.5,0.5,0.0,0.00,7.90
3,2010000004,2010000004,VTS,2010-01-01 00:00:00,Cre,56.1,0.5,0.5,10.0,9.14,76.24
4,2010000005,2010000005,VTS,2010-01-01 00:00:00,CAS,14.5,0.5,0.5,0.0,0.00,15.50


In [4]:
new_y1001_1.isna().sum().compute()

medallion                    0
 hack_license                0
 vendor_id                   0
 rate_code                   0
 store_and_fwd_flag    8292619
 pickup_datetime             0
 dropoff_datetime            0
 passenger_count             0
 trip_time_in_secs           0
 trip_distance               0
 pickup_longitude            0
 pickup_latitude             0
 dropoff_longitude           0
 dropoff_latitude            0
dtype: int64

In [5]:
new_y1001_1_fare.isna().sum().compute()

medallion           0
 hack_license       0
 vendor_id          0
 pickup_datetime    0
 payment_type       0
 fare_amount        0
 surcharge          0
 mta_tax            0
 tip_amount         0
 tolls_amount       0
 total_amount       0
dtype: int64

In [4]:
new_y1001_1=new_y1001_1.fillna(0)
new_y1001_1.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2010000001,2010000001,VTS,1,0,2010-01-01 00:00:00,2010-01-01 00:34:00,1,34.0,14.05,-73.948418,40.724590,-73.926140,40.864761
1,2010000002,2010000002,VTS,1,0,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.0,9.65,-73.997414,40.736156,-73.997833,40.736168
2,2010000003,2010000003,VTS,1,0,2010-01-01 00:00:00,2010-01-01 00:07:00,1,7.0,1.63,-73.967171,40.764236,-73.956299,40.781261
3,2010000004,2010000004,VTS,1,0,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.0,26.61,-73.789757,40.646526,-74.136749,40.601543
4,2010000005,2010000005,VTS,1,0,2010-01-01 00:00:00,2010-01-01 00:28:00,2,28.0,3.15,-73.999550,40.731152,-73.977448,40.763031


In [5]:
new_y1001_1_fare=new_y1001_1_fare.repartition(npartitions=new_y1001_1.npartitions)

In [7]:
new_y1001_1.describe().compute()

,medallion,hack_license,rate_code,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
count,1.486378e+07,1.486378e+07,14863778.0,1.486378e+07,1.486378e+07,1.486378e+07,1.486378e+07,1.486378e+07,1.486378e+07,1.486378e+07
mean,2.010006e+09,2.010013e+09,1.0,1.684686e+00,1.077795e+01,2.628267e+00,-7.263310e+01,4.001195e+01,-7.265785e+01,4.002671e+01
std,3.616893e+03,8.255003e+03,0.0,1.260632e+00,7.643000e+00,3.018446e+00,9.901503e+00,5.628441e+00,9.823565e+00,5.440934e+00
min,2.010000e+09,2.010000e+09,1.0,0.000000e+00,0.000000e+00,0.000000e+00,-7.832167e+02,-3.538407e+03,-2.069479e+03,-2.269701e+00
25%,2.010003e+09,2.010009e+09,1.0,1.000000e+00,6.000000e+00,1.100000e+00,-7.399090e+01,4.073913e+01,-7.399049e+01,4.073872e+01
50%,2.010006e+09,2.010015e+09,1.0,1.000000e+00,9.770000e+00,1.910000e+00,-7.397995e+01,4.075582e+01,-7.397907e+01,4.075590e+01
75%,2.010009e+09,2.010022e+09,1.0,2.000000e+00,1.487000e+01,3.600000e+00,-7.396440e+01,4.076967e+01,-7.396014e+01,4.077055e+01
max,2.010013e+09,2.010031e+09,1.0,2.080000e+02,1.800000e+02,5.000000e+01,8.434618e+01,1.541867e+03,4.276011e+01,1.541867e+03


In [8]:
new_y1001_1_fare.describe().compute()

,medallion,hack_license,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
count,1.486378e+07,1.486378e+07,1.486378e+07,1.486378e+07,1.486378e+07,1.486378e+07,1.486378e+07,1.486378e+07
mean,2.010006e+09,2.010013e+09,9.427919e+00,3.196215e-01,4.706081e-01,6.714118e-01,1.470634e-01,1.106182e+01
std,3.616893e+03,8.255003e+03,7.379477e+00,3.663894e-01,1.937933e-01,1.464261e+00,8.585346e-01,8.594386e+00
min,2.010000e+09,2.010000e+09,2.500000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,2.500000e+00
25%,2.010003e+09,2.010009e+09,5.300000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,6.700000e+00
50%,2.010006e+09,2.010015e+09,7.300000e+00,5.000000e-01,5.000000e-01,0.000000e+00,0.000000e+00,8.900000e+00
75%,2.010009e+09,2.010022e+09,1.090000e+01,1.000000e+00,5.000000e-01,1.100000e+00,0.000000e+00,1.260000e+01
max,2.010013e+09,2.010031e+09,2.000000e+02,1.050000e+01,5.000000e-01,1.000000e+02,2.000000e+01,2.360000e+02


In [9]:
len(new_y1001_1),len(new_y1001_1_fare)

(14863778, 14863778)

In [10]:
new_1=new_y1001_1.merge(new_y1001_1_fare)
new_1.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,2010000064,2010013375,CMT,1,0,2010-01-30 14:52:19,2010-01-30 15:14:30,1,22.18,5.7,...,40.752106,-73.948929,40.805103,Cre,17.3,0.0,0.5,1.2,0.00,19.00
1,2010000421,2010008428,CMT,1,0,2010-01-30 14:52:20,2010-01-30 15:04:06,1,11.77,1.7,...,40.763378,-73.957886,40.777061,Cas,8.5,0.0,0.5,0.0,0.00,9.00
2,2010009804,2010014965,CMT,1,0,2010-01-30 14:52:32,2010-01-30 15:17:25,2,24.88,16.6,...,40.757713,-73.782555,40.644112,Cas,45.0,0.0,0.5,0.0,4.57,50.07
3,2010008115,2010028517,CMT,1,0,2010-01-30 14:52:34,2010-01-30 15:02:03,1,9.48,0.9,...,40.738026,-73.989891,40.730312,Cas,6.5,0.0,0.5,0.0,0.00,7.00
4,2010000710,2010029921,CMT,1,0,2010-01-30 14:52:50,2010-01-30 15:07:38,1,14.80,2.0,...,40.762264,-74.002876,40.760147,Cas,8.9,0.0,0.5,0.0,0.00,9.40


In [11]:
new_1.columns=new_1.columns.map(lambda x: x.replace(" ",""))
new_1.columns

Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'payment_type', 'fare_amount', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount'],
      dtype='object')

In [12]:
with ProgressBar():
    new_1['pickup_time']=new_1['pickup_datetime'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S"),meta=dt)
    new_1['dropoff_time']=new_1['dropoff_datetime'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S"),meta=dt)
    new_11=new_1.set_index('pickup_time')

[########################################] | 100% Completed | 10min 54.3s


In [13]:
with ProgressBar():
    display(new_11.head(20))

[########################################] | 100% Completed | 11min 51.0s


,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_time
pickup_time,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,2010000008,2010000008,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:27:00,1,27.00,9.83,...,-74.002800,40.760498,Cre,25.7,0.5,0.5,0.00,4.57,31.27,2010-01-01 00:27:00
2010-01-01 00:00:00,2010000004,2010000004,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.00,26.61,...,-74.136749,40.601543,Cre,56.1,0.5,0.5,10.00,9.14,76.24,2010-01-01 00:33:00
2010-01-01 00:00:00,2010000003,2010000003,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:07:00,1,7.00,1.63,...,-73.956299,40.781261,CAS,6.9,0.5,0.5,0.00,0.00,7.90,2010-01-01 00:07:00
2010-01-01 00:00:00,2010000009,2010000009,CMT,1,0,2010-01-01 00:00:00,2010-01-01 00:18:13,1,18.22,3.40,...,-73.988342,40.718399,Cas,12.5,0.5,0.5,0.00,0.00,13.50,2010-01-01 00:18:13
2010-01-01 00:00:00,2010000002,2010000002,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.00,9.65,...,-73.997833,40.736168,CAS,27.3,0.5,0.5,0.00,0.00,28.30,2010-01-01 00:33:00
2010-01-01 00:00:00,2010000001,2010000001,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:34:00,1,34.00,14.05,...,-73.926140,40.864761,CAS,34.1,0.5,0.5,0.00,0.00,35.10,2010-01-01 00:34:00
2010-01-01 00:00:00,2010000005,2010000005,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:28:00,2,28.00,3.15,...,-73.977448,40.763031,CAS,14.5,0.5,0.5,0.00,0.00,15.50,2010-01-01 00:28:00
2010-01-01 00:00:00,2010000007,2010000007,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:18:00,3,18.00,4.30,...,-73.957405,40.765686,CAS,13.3,0.5,0.5,0.00,0.00,14.30,2010-01-01 00:18:00
2010-01-01 00:00:00,2010000006,2010000006,VTS,1,NaN,2010-01-01 00:00:00,2010-01-01 00:27:00,1,27.00,11.15,...,-73.861435,40.756256,CAS,27.7,0.5,0.5,0.00,0.00,28.70,2010-01-01 00:27:00


In [14]:
new_11.loc['2010-01-01':'2010-01-02'].head(10)

KeyboardInterrupt: 

In [ ]:
new_11.loc['2010-01-01':'2010-01-02'].head(-10)

In [15]:
new_1.isna().sum().compute()

KeyboardInterrupt: 

In [9]:
taxizone=pd.read_csv("D:/Working_Place/Uber/taxi_zones.csv")

zonelist=[]
for j in range(263):
    numbers=[]
    stri=''
    for i in enumerate(taxizone.loc[j][2]):
        if i[1] in ['0','1','2','3','4','5','6','7','8','9','0','.','-']:
            stri+=i[1]
        else:
            numbers+=[stri]
            stri=''
    while '' in numbers:
        numbers.remove('')
    anumbers=[]
    for i in range(len(numbers)//2):
        a1=float(numbers[2*i])
        a2=float(numbers[2*i+1])
        anumbers+=[(a1,a2)]
    b=Polygon(anumbers)
    zonelist+=[b]
print(len(zonelist))

def findingthezone(point):
    ind=0
    for i in zonelist:
        if point.within(i):
            return(ind)
        else:
            ind+=1
    if ind==263:
        return -1

def ptype(i):   
    if i=='Cash' or i=='CASH':
        return 2
    elif i=='CREDIT' or i=='Credit':
        return 1
    elif i=='Dispute':
        return 4
    else:
        return 3

def Vid(i):
    if i=='CMT':
        return 1
    elif i=='DDS':
        return 3
    else:
        return 2

def ptype11(i):
    if i in ['Cas', 'CAS' ,'CSH']:
        return 2
    elif i in ['CRE', 'Cre' , 'CRD']:
        return 1
    elif i in ['Dis' , 'DIS']:
        return 4
    else:
        return 3

263


In [ ]:
new_1.vendor_id.unique().compute()

In [ ]:
new_1.payment_type.unique().compute()

In [10]:
new_11['vendor_id']=new_11.apply(lambda x:Vid(x.vendor_id),axis=1,meta='int64')
new_11['payment_type']=new_11.apply(lambda x: ptype11(x.payment_type),axis=1,meta='int64')
new_11['pickup_loc']=new_11.apply(lambda x: findingthezone(Point(x.pickup_longitude,x.pickup_latitude)),axis=1,meta='int64')
new_11['dropoff_loc']=new_11.apply(lambda x: findingthezone(Point(x.dropoff_longitude,x.dropoff_latitude)),axis=1,meta='int64')

In [11]:
with ProgressBar():
    display(new_11.head(20))

[########################################] | 100% Completed | 23min 46.9s


,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_time,pickup_loc,dropoff_loc
pickup_time,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,2010000003,2010000003,2,1,0,2010-01-01 00:00:00,2010-01-01 00:07:00,1,7.00,1.63,...,2,6.9,0.5,0.5,0.00,0.00,7.90,2010-01-01 00:07:00,237,236
2010-01-01 00:00:00,2010000004,2010000004,2,1,0,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.00,26.61,...,1,56.1,0.5,0.5,10.00,9.14,76.24,2010-01-01 00:33:00,136,19
2010-01-01 00:00:00,2010000005,2010000005,2,1,0,2010-01-01 00:00:00,2010-01-01 00:28:00,2,28.00,3.15,...,2,14.5,0.5,0.5,0.00,0.00,15.50,2010-01-01 00:28:00,114,160
2010-01-01 00:00:00,2010000002,2010000002,2,1,0,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.00,9.65,...,2,27.3,0.5,0.5,0.00,0.00,28.30,2010-01-01 00:33:00,113,251
2010-01-01 00:00:00,2010000001,2010000001,2,1,0,2010-01-01 00:00:00,2010-01-01 00:34:00,1,34.00,14.05,...,2,34.1,0.5,0.5,0.00,0.00,35.10,2010-01-01 00:34:00,112,128
2010-01-01 00:00:00,2010000006,2010000006,2,1,0,2010-01-01 00:00:00,2010-01-01 00:27:00,1,27.00,11.15,...,2,27.7,0.5,0.5,0.00,0.00,28.70,2010-01-01 00:27:00,234,173
2010-01-01 00:00:00,2010000007,2010000007,2,1,0,2010-01-01 00:00:00,2010-01-01 00:18:00,3,18.00,4.30,...,2,13.3,0.5,0.5,0.00,0.00,14.30,2010-01-01 00:18:00,158,138
2010-01-01 00:00:00,2010000009,2010000009,1,1,0,2010-01-01 00:00:00,2010-01-01 00:18:13,1,18.22,3.40,...,2,12.5,0.5,0.5,0.00,0.00,13.50,2010-01-01 00:18:13,248,151
2010-01-01 00:00:00,2010000008,2010000008,2,1,0,2010-01-01 00:00:00,2010-01-01 00:27:00,1,27.00,9.83,...,1,25.7,0.5,0.5,0.00,4.57,31.27,2010-01-01 00:27:00,145,248


In [15]:
with ProgressBar():
    new_11['pickup_datetime']=new_11['pickup_datetime'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S"),meta=dt)
    new_11['dropoff_datetime']=new_11['dropoff_datetime'].apply(lambda x: dt.strptime(x, "%Y-%m-%d %H:%M:%S"),meta=dt)

In [17]:
with ProgressBar():
    display(new_11.head(10))

[########################################] | 100% Completed | 24min 44.2s


,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_time,pickup_loc,dropoff_loc
pickup_time,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,2010000005,2010000005,2,1,0,2010-01-01 00:00:00,2010-01-01 00:28:00,2,28.00,3.15,...,2,14.5,0.5,0.5,0.0,0.00,15.50,2010-01-01 00:28:00,114,160
2010-01-01 00:00:00,2010000009,2010000009,1,1,0,2010-01-01 00:00:00,2010-01-01 00:18:13,1,18.22,3.40,...,2,12.5,0.5,0.5,0.0,0.00,13.50,2010-01-01 00:18:13,248,151
2010-01-01 00:00:00,2010000008,2010000008,2,1,0,2010-01-01 00:00:00,2010-01-01 00:27:00,1,27.00,9.83,...,1,25.7,0.5,0.5,0.0,4.57,31.27,2010-01-01 00:27:00,145,248
2010-01-01 00:00:00,2010000007,2010000007,2,1,0,2010-01-01 00:00:00,2010-01-01 00:18:00,3,18.00,4.30,...,2,13.3,0.5,0.5,0.0,0.00,14.30,2010-01-01 00:18:00,158,138
2010-01-01 00:00:00,2010000006,2010000006,2,1,0,2010-01-01 00:00:00,2010-01-01 00:27:00,1,27.00,11.15,...,2,27.7,0.5,0.5,0.0,0.00,28.70,2010-01-01 00:27:00,234,173
2010-01-01 00:00:00,2010000004,2010000004,2,1,0,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.00,26.61,...,1,56.1,0.5,0.5,10.0,9.14,76.24,2010-01-01 00:33:00,136,19
2010-01-01 00:00:00,2010000003,2010000003,2,1,0,2010-01-01 00:00:00,2010-01-01 00:07:00,1,7.00,1.63,...,2,6.9,0.5,0.5,0.0,0.00,7.90,2010-01-01 00:07:00,237,236
2010-01-01 00:00:00,2010000001,2010000001,2,1,0,2010-01-01 00:00:00,2010-01-01 00:34:00,1,34.00,14.05,...,2,34.1,0.5,0.5,0.0,0.00,35.10,2010-01-01 00:34:00,112,128
2010-01-01 00:00:00,2010000002,2010000002,2,1,0,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.00,9.65,...,2,27.3,0.5,0.5,0.0,0.00,28.30,2010-01-01 00:33:00,113,251


In [27]:
kk=dt(2010,1,2)
print(kk)
print(kk.date())

2010-01-02 00:00:00
2010-01-02


In [30]:
with ProgressBar():
    start_day=new_11['pickup_datetime'].apply(lambda x: x.date(),meta=dt)
    display(start_day.head())

[########################################] | 100% Completed | 27min  5.1s


pickup_time
2010-01-01    2010-01-01
2010-01-01    2010-01-01
2010-01-01    2010-01-01
2010-01-01    2010-01-01
2010-01-01    2010-01-01
dtype: object

In [31]:
with ProgressBar():
    new_11['start_day']=start_day
    display(new_11.head(10))

[########################################] | 100% Completed | 24min 59.3s


,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_time,pickup_loc,dropoff_loc,start_day
pickup_time,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,2010000003,2010000003,2,1,0,2010-01-01 00:00:00,2010-01-01 00:07:00,1,7.00,1.63,...,6.9,0.5,0.5,0.0,0.00,7.90,2010-01-01 00:07:00,237,236,2010-01-01
2010-01-01 00:00:00,2010000007,2010000007,2,1,0,2010-01-01 00:00:00,2010-01-01 00:18:00,3,18.00,4.30,...,13.3,0.5,0.5,0.0,0.00,14.30,2010-01-01 00:18:00,158,138,2010-01-01
2010-01-01 00:00:00,2010000001,2010000001,2,1,0,2010-01-01 00:00:00,2010-01-01 00:34:00,1,34.00,14.05,...,34.1,0.5,0.5,0.0,0.00,35.10,2010-01-01 00:34:00,112,128,2010-01-01
2010-01-01 00:00:00,2010000006,2010000006,2,1,0,2010-01-01 00:00:00,2010-01-01 00:27:00,1,27.00,11.15,...,27.7,0.5,0.5,0.0,0.00,28.70,2010-01-01 00:27:00,234,173,2010-01-01
2010-01-01 00:00:00,2010000009,2010000009,1,1,0,2010-01-01 00:00:00,2010-01-01 00:18:13,1,18.22,3.40,...,12.5,0.5,0.5,0.0,0.00,13.50,2010-01-01 00:18:13,248,151,2010-01-01
2010-01-01 00:00:00,2010000002,2010000002,2,1,0,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.00,9.65,...,27.3,0.5,0.5,0.0,0.00,28.30,2010-01-01 00:33:00,113,251,2010-01-01
2010-01-01 00:00:00,2010000004,2010000004,2,1,0,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.00,26.61,...,56.1,0.5,0.5,10.0,9.14,76.24,2010-01-01 00:33:00,136,19,2010-01-01
2010-01-01 00:00:00,2010000008,2010000008,2,1,0,2010-01-01 00:00:00,2010-01-01 00:27:00,1,27.00,9.83,...,25.7,0.5,0.5,0.0,4.57,31.27,2010-01-01 00:27:00,145,248,2010-01-01
2010-01-01 00:00:00,2010000005,2010000005,2,1,0,2010-01-01 00:00:00,2010-01-01 00:28:00,2,28.00,3.15,...,14.5,0.5,0.5,0.0,0.00,15.50,2010-01-01 00:28:00,114,160,2010-01-01


In [40]:
with ProgressBar():
    new_11['tip_rate']=new_11['tip_amount']/new_11['fare_amount']
    display(new_11.head())

[########################################] | 100% Completed | 10hr 13min 38.2s


,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_time,pickup_loc,dropoff_loc,start_day,tip_rate
pickup_time,,,,,,,,,,,,,,,,,,,,,
2010-01-01,2010000001,2010000001,2,1,0,2010-01-01,2010-01-01 00:34:00,1,34.0,14.05,...,0.5,0.5,0.0,0.00,35.10,2010-01-01 00:34:00,112,128,2010-01-01,0.0
2010-01-01,2010000003,2010000003,2,1,0,2010-01-01,2010-01-01 00:07:00,1,7.0,1.63,...,0.5,0.5,0.0,0.00,7.90,2010-01-01 00:07:00,237,236,2010-01-01,0.0
2010-01-01,2010000005,2010000005,2,1,0,2010-01-01,2010-01-01 00:28:00,2,28.0,3.15,...,0.5,0.5,0.0,0.00,15.50,2010-01-01 00:28:00,114,160,2010-01-01,0.0
2010-01-01,2010000006,2010000006,2,1,0,2010-01-01,2010-01-01 00:27:00,1,27.0,11.15,...,0.5,0.5,0.0,0.00,28.70,2010-01-01 00:27:00,234,173,2010-01-01,0.0
2010-01-01,2010000008,2010000008,2,1,0,2010-01-01,2010-01-01 00:27:00,1,27.0,9.83,...,0.5,0.5,0.0,4.57,31.27,2010-01-01 00:27:00,145,248,2010-01-01,0.0


In [41]:
with ProgressBar():
    aaa=new_11.groupby(['medallion','hack_license','start_day']).agg({'passenger_count':['count','sum','mean','min','max'],
                                                             'trip_time_in_secs':['sum','mean','min','max'],
                                                             'trip_distance':['sum','mean','min','max'],
                                                             'tip_amount':['sum','mean','min','max'],
                                                             'fare_amount':['sum','mean','min','max'],
                                                                     'tip_rate':['mean','min','max']})
    display(aaa.compute())

[########################################] | 100% Completed | 11hr 41min 29.7s


passenger_count                        \
                                             count sum      mean min max   
medallion  hack_license start_day                                          
2010000001 2010000001   2010-01-01               8  13  1.625000   1   3   
                        2010-01-02              13  32  2.461538   1   4   
                        2010-01-03               1   1  1.000000   1   1   
                        2010-01-04               4   8  2.000000   1   4   
                        2010-01-05               4   7  1.750000   1   3   
...                                            ...  ..       ...  ..  ..   
2010013338 2010030492   2010-01-30              27  27  1.000000   1   1   
2010013339 2010025814   2010-01-30               2   2  1.000000   1   1   
2010013340 2010003087   2010-01-30              12  60  5.000000   5   5   
                        2010-01-31               7  35  5.000000   5   5   
2010013341 2010030651   2010-01-31               1   2  2.000000   2   2   

                                   trip_time_in_secs                           \
                                                 sum       mean    min    max   
medallion  hack_license start_day                                               
2010000001 2010000001   2010-01-01            155.00  19.375000   4.00  44.00   
                        2010-01-02            195.00  15.000000   5.00  53.00   
                        2010-01-03              7.00   7.000000   7.00   7.00   
                        2010-01-04            108.00  27.000000   4.00  42.00   
                        2010-01-05            119.00  29.750000  19.00  44.00   
...                                              ...        ...    ...    ...   
2010013338 2010030492   2010-01-30            291.43  10.793704   3.60  31.09   
2010013339 2010025814   2010-01-30              1.00   0.500000   0.00   1.00   
2010013340 2010003087   2010-01-30            149.00  12.416667   4.00  36.00   
                        2010-01-31            102.00  14.571429  10.00  20.00   
2010013341 2010030651   2010-01-31              1.63   1.630000   1.63   1.63   

                                   trip_distance  ... tip_amount             \
                                             sum  ...       mean  min   max   
medallion  hack_license start_day                 ...                         
2010000001 2010000001   2010-01-01         64.11  ...   1.125000  0.0   6.0   
                        2010-01-02         56.57  ...   1.750000  0.0   8.5   
                        2010-01-03          1.90  ...   0.000000  0.0   0.0   
                        2010-01-04         58.66  ...   2.250000  0.0   9.0   
                        2010-01-05         60.74  ...   0.000000  0.0   0.0   
...                                          ...  ...        ...  ...   ...   
2010013338 2010030492   2010-01-30         52.60  ...   0.360000  0.0   2.0   
2010013339 2010025814   2010-01-30          0.00  ...   0.000000  0.0   0.0   
2010013340 2010003087   2010-01-30         26.41  ...   2.220000  0.0  10.0   
                        2010-01-31         18.97  ...   1.285714  0.0   3.0   
2010013341 2010030651   2010-01-31          0.00  ...   0.000000  0.0   0.0   

                                   fare_amount                           \
                                           sum       mean    min    max   
medallion  hack_license start_day                                         
2010000001 2010000001   2010-01-01      163.30  20.412500   4.50  45.00   
                        2010-01-02      170.70  13.130769   4.50  45.00   
                        2010-01-03        6.90   6.900000   6.90   6.90   
                        2010-01-04      140.20  35.050000   4.50  45.70   
                        2010-01-05      148.70  37.175000  13.70  45.00   
...                                        ...        ...    ...    ...   
2010013338 2010030492   2010-01-30      213.90   7.92

In [54]:
aaa.npartitions

1

In [46]:
new_11.dtypes

medallion               int64
hack_license            int64
vendor_id             float64
rate_code               int64
store_and_fwd_flag     object
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
trip_time_in_secs     float64
trip_distance         float64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
payment_type          float64
fare_amount           float64
surcharge             float64
mta_tax               float64
tip_amount            float64
tolls_amount          float64
total_amount          float64
dropoff_time           object
pickup_loc            float64
dropoff_loc           float64
start_day              object
tip_rate              float64
dtype: object

In [51]:
with ProgressBar():
    new_111=new_11.reset_index().astype({'pickup_datetime':'str','dropoff_datetime':'str','pickup_time':'str','dropoff_time':'str'})
    display(new_111.head())

[########################################] | 100% Completed | 26min 17.1s


,pickup_time,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,...,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_time,pickup_loc,dropoff_loc,start_day,tip_rate
0,2010-01-01 00:00:00,2010000008,2010000008,2,1,0,2010-01-01 00:00:00,2010-01-01 00:27:00,1,27.0,...,0.5,0.5,0.0,4.57,31.27,2010-01-01 00:27:00,145,248,2010-01-01,0.000000
1,2010-01-01 00:00:00,2010000005,2010000005,2,1,0,2010-01-01 00:00:00,2010-01-01 00:28:00,2,28.0,...,0.5,0.5,0.0,0.00,15.50,2010-01-01 00:28:00,114,160,2010-01-01,0.000000
2,2010-01-01 00:00:00,2010000001,2010000001,2,1,0,2010-01-01 00:00:00,2010-01-01 00:34:00,1,34.0,...,0.5,0.5,0.0,0.00,35.10,2010-01-01 00:34:00,112,128,2010-01-01,0.000000
3,2010-01-01 00:00:00,2010000003,2010000003,2,1,0,2010-01-01 00:00:00,2010-01-01 00:07:00,1,7.0,...,0.5,0.5,0.0,0.00,7.90,2010-01-01 00:07:00,237,236,2010-01-01,0.000000
4,2010-01-01 00:00:00,2010000004,2010000004,2,1,0,2010-01-01 00:00:00,2010-01-01 00:33:00,1,33.0,...,0.5,0.5,10.0,9.14,76.24,2010-01-01 00:33:00,136,19,2010-01-01,0.178253


In [52]:
new_111.dtypes

pickup_time            object
medallion               int64
hack_license            int64
vendor_id             float64
rate_code               int64
store_and_fwd_flag     object
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
trip_time_in_secs     float64
trip_distance         float64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
payment_type          float64
fare_amount           float64
surcharge             float64
mta_tax               float64
tip_amount            float64
tolls_amount          float64
total_amount          float64
dropoff_time           object
pickup_loc            float64
dropoff_loc           float64
start_day              object
tip_rate              float64
dtype: object

In [55]:
with ProgressBar():
    new_111.to_csv('new_y1001_csv/new_y1001-*.csv')

[########################################] | 100% Completed | 12hr 42min 45.3s


In [56]:
with ProgressBar():
    aaa.to_csv('new_y1001_csv/new_y1001_summary-*.csv')

[########################################] | 100% Completed | 11hr  9min  3.1s


In [ ]:
with ProgressBar():
    count=0
    for i in aa.index:
        count+=1
        bb=new_11[(new_11.medallion==i[0]) & (new_11.hack_license==i[1])]
        display(bb.head())
        if count==10:
            break

In [ ]:
with ProgressBar():
    display(bb.head())

In [ ]:
with ProgressBar():
    bb=new_11[(new_11.medallion==2010000001) & (new_11.hack_license==2010016645)]
    display(bb.head())

In [ ]:
with ProgressBar():
    display(bb.loc['2010-01-02':'2010-01-02'].compute())

In [ ]:
with ProgressBar():
    display(bb.loc['2010-01-02 14:47:00':'2010-01-02 14:47:00'].compute())

In [ ]:
with ProgressBar():
    cc=bb.index.to_series()
    display(cc.head())

In [ ]:
with ProgressBar():
    bbb=bb.loc['2010-01-02':'2010-01-02']
    #bbb=bbb.reset_index()
    display(bbb.head())